## 1 Data Preprocessing

In [1]:
import re
import requests
import bs4
import pandas as pd
import geopandas as gpd
import math
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import sqlalchemy

In [2]:
import warnings
warnings.filterwarnings("ignore")

### 1.1 Preprocessing Yellow taxi trips data

### 1.1.1 downloading data from web


In [3]:
TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Retrieves yellow taxi trip records from the given website. 
def get_taxi_html(): 
    response = requests.get(TAXI_URL)
    html = response.content
    return html

# Send HTTP GET requests to the website, and parse the HTML content and extract links. 
def find_taxi_parquet_links(): 
    taxi_html = get_taxi_html()
    soup = bs4.BeautifulSoup(taxi_html, "html.parser")
    # print(soup.prettify())
    p_link = []
    for item in soup.find_all('a'):
        if "yellow taxi trip records" in item.text.lower():
            p_link.append(item.get('href'))  
            
    return p_link

In [4]:
#uni test


In [5]:
# Downloads the data from the extracted links as Parquet files
def download_parquet(url_list):
    i = 0
    for url in url_list:
        i += 1
        r = requests.get(url, allow_redirects=True)
        open(f'taxi{i}.parquet', 'wb').write(r.content)
        print(f"write {i} successfully")
        

In [6]:
url_list = find_taxi_parquet_links()

In [7]:
len(url_list)

169

In [8]:
for link in url_list:
    print(link)

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-07.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-08.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-09.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-10.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-11.parquet
https://d37ci6vzurychx.cloudfront.net/tr

In [9]:
## have been finished and only need to execute one time
# download_parquet(url_list)

### 1.1.2 Cleaning & filtering

In [10]:
### Sampling & downloading for yellow taxi trips

In [11]:
# Read all parquet files
import random
n = len(url_list)
# from June 2015 -> Jan 2009
i = 89
print("start reading!")
tmp_pq = pd.read_parquet(f'taxi{i}.parquet')
ye_taxi_pq = tmp_pq.sample(n=250, random_state=1)
while(i<=166):
    print(f"finish reading {i}")
    i += 1
    tmp_pq = pd.read_parquet(f'taxi{i}.parquet')
    # sample: each month keep 2500 data and combine them to a roughly 200000-row dataframe
    sample_tmp_pq = tmp_pq.sample(n=4000, random_state=1)
    ye_taxi_pq = pd.concat([ye_taxi_pq, sample_tmp_pq])
ye_taxi_pq

start reading!
finish reading 89
finish reading 90
finish reading 91
finish reading 92
finish reading 93
finish reading 94
finish reading 95
finish reading 96
finish reading 97
finish reading 98
finish reading 99
finish reading 100
finish reading 101
finish reading 102
finish reading 103
finish reading 104
finish reading 105
finish reading 106
finish reading 107
finish reading 108
finish reading 109
finish reading 110
finish reading 111
finish reading 112
finish reading 113
finish reading 114
finish reading 115
finish reading 116
finish reading 117
finish reading 118
finish reading 119
finish reading 120
finish reading 121
finish reading 122
finish reading 123
finish reading 124
finish reading 125
finish reading 126
finish reading 127
finish reading 128
finish reading 129
finish reading 130
finish reading 131
finish reading 132
finish reading 133
finish reading 134
finish reading 135
finish reading 136
finish reading 137
finish reading 138
finish reading 139
finish reading 140
finish r

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,Tip_Amt,Tolls_Amt,Total_Amt
488064,2.0,2015-06-02 08:37:30,2015-06-02 08:47:15,1.0,1.10,1.0,N,234.0,113.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5073891,1.0,2015-06-12 22:59:41,2015-06-12 23:12:10,1.0,1.90,1.0,N,137.0,4.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8611696,2.0,2015-06-21 15:23:01,2015-06-21 15:26:14,2.0,0.75,1.0,N,158.0,68.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3777133,1.0,2015-06-09 23:55:41,2015-06-10 00:01:48,1.0,1.40,1.0,N,236.0,238.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5052093,1.0,2015-06-12 22:56:36,2015-06-12 23:25:27,1.0,2.10,1.0,N,163.0,170.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089537,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.754909,NaN,0.0,-73.994973,40.760164,Cash,4.5,0.0,0.0,5.0
9202173,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.740975,NaN,0.0,-74.005627,40.733261,Cash,4.1,0.0,0.0,5.6
3675040,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.771276,NaN,NaN,-73.980070,40.779761,CASH,7.3,0.0,0.0,7.8
5867047,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.779477,NaN,NaN,-73.975965,40.792175,Credit,5.3,0.5,0.0,6.3


In [12]:
# ye_taxi_pq.info()

In [13]:
zones_gdf = gpd.read_file('taxi_zones.shp')

In [14]:
zones_gdf

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


In [15]:
# use the “center” of the zones (polygons) as a point to look up the coordinates

In [16]:
zones_gdf = zones_gdf.to_crs(4326)
zones_gdf['lon'] = zones_gdf.centroid.x  
zones_gdf['lat'] = zones_gdf.centroid.y

In [17]:
zones_gdf

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,lon,lat
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",-74.174000,40.691831
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...",-73.847422,40.864474
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",-73.976968,40.723752
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...",-74.188484,40.552659
...,...,...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.85107 40.91037, -73.85207 40.909...",-73.852215,40.897932
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((-73.90175 40.76078, -73.90147 40.759...",-73.906306,40.744235
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((-74.01333 40.70503, -74.01327 40.704...",-74.013023,40.709139
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ...",-73.946510,40.775932


In [18]:
len(ye_taxi_pq)

312250

In [19]:
# only keeping columns needed
# according to the sample data
cleaned_df = ye_taxi_pq[["fare_amount", "tpep_pickup_datetime", "PULocationID", "DOLocationID", "passenger_count", "tip_amount"]]

In [20]:
## drop invalid location ID
pickup_valid = cleaned_df["PULocationID"] < 264
dropof_valid = cleaned_df["DOLocationID"] < 264

cleaned_df = cleaned_df[pickup_valid & dropof_valid]

In [21]:
len(cleaned_df)

210647

In [22]:
# rename column
cleaned_df.rename(columns={"tpep_pickup_datetime": "pickup_datetime"}, inplace=True)
cleaned_df.rename(columns={"tip_amount": "tips"}, inplace=True)
cleaned_df

,fare_amount,pickup_datetime,PULocationID,DOLocationID,passenger_count,tips
488064,7.5,2015-06-02 08:37:30,234.0,113.0,1.0,0.00
5073891,10.0,2015-06-12 22:59:41,137.0,4.0,1.0,0.00
8611696,4.5,2015-06-21 15:23:01,158.0,68.0,2.0,0.00
3777133,7.0,2015-06-09 23:55:41,236.0,238.0,1.0,1.00
5052093,17.5,2015-06-12 22:56:36,163.0,170.0,1.0,0.00
...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,142.0,79.0,5.0,0.00
8492593,12.1,2011-12-17 10:13:12,87.0,233.0,1.0,2.00
3460249,6.9,2011-12-07 20:13:00,237.0,75.0,1.0,0.00
1159551,7.7,2011-12-03 03:52:42,79.0,186.0,1.0,2.17


In [23]:
# add latitude and longtitude
pid = cleaned_df["PULocationID"]-1
cleaned_df["pickup_longtitude"] = zones_gdf["lon"][pid].values
cleaned_df["pickup_latitude"] = zones_gdf["lat"][pid].values

did = cleaned_df["DOLocationID"]-1
cleaned_df["dropoff_longtitude"] = zones_gdf["lon"][did].values
cleaned_df["dropoff_latitude"] = zones_gdf["lat"][did].values

In [24]:
cleaned_df = cleaned_df.drop(["PULocationID", "DOLocationID"], axis=1)

In [25]:
copy_df = cleaned_df.copy()
copy_df

,fare_amount,pickup_datetime,passenger_count,tips,pickup_longtitude,pickup_latitude,dropoff_longtitude,dropoff_latitude
488064,7.5,2015-06-02 08:37:30,1.0,0.00,-73.990458,40.740337,-73.994305,40.732579
5073891,10.0,2015-06-12 22:59:41,1.0,0.00,-73.976495,40.740439,-73.976968,40.723752
8611696,4.5,2015-06-21 15:23:01,2.0,0.00,-74.008984,40.735035,-73.999917,40.748428
3777133,7.0,2015-06-09 23:55:41,1.0,1.00,-73.957012,40.780436,-73.973049,40.791705
5052093,17.5,2015-06-12 22:56:36,1.0,0.00,-73.977569,40.764421,-73.978492,40.747746
...,...,...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,5.0,0.00,-73.981532,40.773633,-73.985937,40.727620
8492593,12.1,2011-12-17 10:13:12,1.0,2.00,-74.007496,40.706808,-73.970443,40.749914
3460249,6.9,2011-12-07 20:13:00,1.0,0.00,-73.965635,40.768615,-73.945750,40.790011
1159551,7.7,2011-12-03 03:52:42,1.0,2.17,-73.985937,40.727620,-73.992438,40.748497


In [26]:
# just keep valid lon&lat
# removing trips that start and/or end outside of the
# following latitude/longitude coordinate box:
# (40.560445, -74.242330) and (40.908524, -73.717047).

valid_pick_lon = (cleaned_df["pickup_longtitude"] > -74.242330) & (cleaned_df["pickup_longtitude"] < -73.717047)
valid_pick_lat = (cleaned_df["pickup_latitude"] > 40.560445) & (cleaned_df["pickup_latitude"] < 40.908524)

valid_drop_lon = (cleaned_df["dropoff_longtitude"] > -74.242330) & (cleaned_df["dropoff_longtitude"] < -73.717047)
valid_drop_lat = (cleaned_df["dropoff_latitude"] > 40.560445) & (cleaned_df["dropoff_latitude"] < 40.908524)

cleaned_df = cleaned_df[valid_pick_lon & valid_pick_lat & valid_drop_lon & valid_drop_lat]

In [27]:
cleaned_df

,fare_amount,pickup_datetime,passenger_count,tips,pickup_longtitude,pickup_latitude,dropoff_longtitude,dropoff_latitude
488064,7.5,2015-06-02 08:37:30,1.0,0.00,-73.990458,40.740337,-73.994305,40.732579
5073891,10.0,2015-06-12 22:59:41,1.0,0.00,-73.976495,40.740439,-73.976968,40.723752
8611696,4.5,2015-06-21 15:23:01,2.0,0.00,-74.008984,40.735035,-73.999917,40.748428
3777133,7.0,2015-06-09 23:55:41,1.0,1.00,-73.957012,40.780436,-73.973049,40.791705
5052093,17.5,2015-06-12 22:56:36,1.0,0.00,-73.977569,40.764421,-73.978492,40.747746
...,...,...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,5.0,0.00,-73.981532,40.773633,-73.985937,40.727620
8492593,12.1,2011-12-17 10:13:12,1.0,2.00,-74.007496,40.706808,-73.970443,40.749914
3460249,6.9,2011-12-07 20:13:00,1.0,0.00,-73.965635,40.768615,-73.945750,40.790011
1159551,7.7,2011-12-03 03:52:42,1.0,2.17,-73.985937,40.727620,-73.992438,40.748497


In [28]:
# drop NaN
cleaned_df = cleaned_df.dropna()
len(cleaned_df)

210628

In [29]:
## for yellow taxi trips
cleaned_df

,fare_amount,pickup_datetime,passenger_count,tips,pickup_longtitude,pickup_latitude,dropoff_longtitude,dropoff_latitude
488064,7.5,2015-06-02 08:37:30,1.0,0.00,-73.990458,40.740337,-73.994305,40.732579
5073891,10.0,2015-06-12 22:59:41,1.0,0.00,-73.976495,40.740439,-73.976968,40.723752
8611696,4.5,2015-06-21 15:23:01,2.0,0.00,-74.008984,40.735035,-73.999917,40.748428
3777133,7.0,2015-06-09 23:55:41,1.0,1.00,-73.957012,40.780436,-73.973049,40.791705
5052093,17.5,2015-06-12 22:56:36,1.0,0.00,-73.977569,40.764421,-73.978492,40.747746
...,...,...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,5.0,0.00,-73.981532,40.773633,-73.985937,40.727620
8492593,12.1,2011-12-17 10:13:12,1.0,2.00,-74.007496,40.706808,-73.970443,40.749914
3460249,6.9,2011-12-07 20:13:00,1.0,0.00,-73.965635,40.768615,-73.945750,40.790011
1159551,7.7,2011-12-03 03:52:42,1.0,2.17,-73.985937,40.727620,-73.992438,40.748497


In [30]:
# date to time columns
def set_times(renamed_data):
    renamed_data["year"] = renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").year )
    renamed_data["month"] = renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").month )
    renamed_data["day"] = renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").day )
    renamed_data["hour"] = renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").hour )
    
    return renamed_data

set_times(cleaned_df)

,fare_amount,pickup_datetime,passenger_count,tips,pickup_longtitude,pickup_latitude,dropoff_longtitude,dropoff_latitude,year,month,day,hour
488064,7.5,2015-06-02 08:37:30,1.0,0.00,-73.990458,40.740337,-73.994305,40.732579,2015,6,2,8
5073891,10.0,2015-06-12 22:59:41,1.0,0.00,-73.976495,40.740439,-73.976968,40.723752,2015,6,12,22
8611696,4.5,2015-06-21 15:23:01,2.0,0.00,-74.008984,40.735035,-73.999917,40.748428,2015,6,21,15
3777133,7.0,2015-06-09 23:55:41,1.0,1.00,-73.957012,40.780436,-73.973049,40.791705,2015,6,9,23
5052093,17.5,2015-06-12 22:56:36,1.0,0.00,-73.977569,40.764421,-73.978492,40.747746,2015,6,12,22
...,...,...,...,...,...,...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,5.0,0.00,-73.981532,40.773633,-73.985937,40.727620,2011,12,23,19
8492593,12.1,2011-12-17 10:13:12,1.0,2.00,-74.007496,40.706808,-73.970443,40.749914,2011,12,17,10
3460249,6.9,2011-12-07 20:13:00,1.0,0.00,-73.965635,40.768615,-73.945750,40.790011,2011,12,7,20
1159551,7.7,2011-12-03 03:52:42,1.0,2.17,-73.985937,40.727620,-73.992438,40.748497,2011,12,3,3


In [31]:
cleaned_df

,fare_amount,pickup_datetime,passenger_count,tips,pickup_longtitude,pickup_latitude,dropoff_longtitude,dropoff_latitude,year,month,day,hour
488064,7.5,2015-06-02 08:37:30,1.0,0.00,-73.990458,40.740337,-73.994305,40.732579,2015,6,2,8
5073891,10.0,2015-06-12 22:59:41,1.0,0.00,-73.976495,40.740439,-73.976968,40.723752,2015,6,12,22
8611696,4.5,2015-06-21 15:23:01,2.0,0.00,-74.008984,40.735035,-73.999917,40.748428,2015,6,21,15
3777133,7.0,2015-06-09 23:55:41,1.0,1.00,-73.957012,40.780436,-73.973049,40.791705,2015,6,9,23
5052093,17.5,2015-06-12 22:56:36,1.0,0.00,-73.977569,40.764421,-73.978492,40.747746,2015,6,12,22
...,...,...,...,...,...,...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,5.0,0.00,-73.981532,40.773633,-73.985937,40.727620,2011,12,23,19
8492593,12.1,2011-12-17 10:13:12,1.0,2.00,-74.007496,40.706808,-73.970443,40.749914,2011,12,17,10
3460249,6.9,2011-12-07 20:13:00,1.0,0.00,-73.965635,40.768615,-73.945750,40.790011,2011,12,7,20
1159551,7.7,2011-12-03 03:52:42,1.0,2.17,-73.985937,40.727620,-73.992438,40.748497,2011,12,3,3


### 1.1.3 Calculating missing data for yellow taxi trips

In [32]:
# for yellow taxi trips
## calculate distance between pickup and dropoff
## by using the Haversine Formula

def to_dist(lat1, lon1, lat2, lon2):    
    R = 6371  # radius of the Earth in kilometers
    dLat = lat2.apply(math.radians) - lat1.apply(math.radians)
    dLon = lon2.apply(math.radians) - lon1.apply(math.radians)
    lat1 = lat1.apply(math.radians)
    lat2 = lat2.apply(math.radians)
    a = (dLat / 2).apply(math.sin) ** 2 + (dLat / 2).apply(math.sin) ** 2 * lat1.apply(math.cos) * lat2.apply(math.cos)
    c = 2 * np.arctan2(a.apply(math.sqrt), (1 - a).apply(math.sqrt))    
#     print("dlat", dLat, "\ndlon", dLon, "\nlat1", lat1, "\nlat2", lat2, "\na", a, "\nc", c)
#     print(type(dLat))
    return R * c

In [33]:
# cleaned_df["distance"] = to_dist(cleaned_df["pickup_latitude"], cleaned_df["pickup_longitude"], cleaned_df["dropoff_latitude"], cleaned_df["dropoff_longitude"])
cleaned_df["distance"] = to_dist(cleaned_df["pickup_latitude"].astype(float), cleaned_df["pickup_longtitude"].astype(float), cleaned_df["dropoff_latitude"].astype(float), cleaned_df["dropoff_longtitude"].astype(float))
cleaned_df    

,fare_amount,pickup_datetime,passenger_count,tips,pickup_longtitude,pickup_latitude,dropoff_longtitude,dropoff_latitude,year,month,day,hour,distance
488064,7.5,2015-06-02 08:37:30,1.0,0.00,-73.990458,40.740337,-73.994305,40.732579,2015,6,2,8,1.082373
5073891,10.0,2015-06-12 22:59:41,1.0,0.00,-73.976495,40.740439,-73.976968,40.723752,2015,6,12,22,2.328042
8611696,4.5,2015-06-21 15:23:01,2.0,0.00,-74.008984,40.735035,-73.999917,40.748428,2015,6,21,15,1.868290
3777133,7.0,2015-06-09 23:55:41,1.0,1.00,-73.957012,40.780436,-73.973049,40.791705,2015,6,9,23,1.571644
5052093,17.5,2015-06-12 22:56:36,1.0,0.00,-73.977569,40.764421,-73.978492,40.747746,2015,6,12,22,2.326170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11735739,14.9,2011-12-23 19:56:00,5.0,0.00,-73.981532,40.773633,-73.985937,40.727620,2011,12,23,19,6.418804
8492593,12.1,2011-12-17 10:13:12,1.0,2.00,-74.007496,40.706808,-73.970443,40.749914,2011,12,17,10,6.013948
3460249,6.9,2011-12-07 20:13:00,1.0,0.00,-73.965635,40.768615,-73.945750,40.790011,2011,12,7,20,2.984202
1159551,7.7,2011-12-03 03:52:42,1.0,2.17,-73.985937,40.727620,-73.992438,40.748497,2011,12,3,3,2.912530


### 1.2 Preprocessing Weather data

In [34]:
# read weather
i = 2009
weather_dfs = [pd.read_csv(f'{i}_weather.csv')]
print(f"finish reading weather in {i}")
while(i < 2015):
    i += 1
    tmp_pq = pd.read_csv(f'{i}_weather.csv')
    weather_dfs.append(tmp_pq) 
#    = pd.concat([weather_df, tmp_pq])
    print(f"finish reading weather in {i}")
weather_dfs = pd.concat(weather_dfs, ignore_index = True)
weather_dfs

finish reading weather in 2009
finish reading weather in 2010
finish reading weather in 2011
finish reading weather in 2012
finish reading weather in 2013
finish reading weather in 2014
finish reading weather in 2015


,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,SOURCE,HourlyAltimeterSetting,HourlyDewPointTemperature,...,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupEquipment,BackupLatitude,BackupLongitude,BackupName,WindEquipmentChangeDate
0,72505394728,2009-01-01T00:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.01,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
1,72505394728,2009-01-01T01:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.03,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
2,72505394728,2009-01-01T02:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.07,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
3,72505394728,2009-01-01T03:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.09,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
4,72505394728,2009-01-01T04:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.12,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77967,72505394728,2015-12-31T21:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,6,30.12,26.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
77968,72505394728,2015-12-31T22:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,7,30.11,28.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
77969,72505394728,2015-12-31T23:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,7,30.1,26.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
77970,72505394728,2015-12-31T23:59:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",SOD,6,NaN,NaN,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18


In [35]:
def clean_weather_data_daily(weather_df):
    #add related daily columns
    tmp_df2 = pd.merge(weather_df['DATE'], weather_df['DailyAverageWindSpeed'], left_index=True, right_index=True)
    tmp_df2 = weather_df[['DATE', 'DailyAverageWindSpeed']]  
    tmp_df2 = tmp_df2.rename(columns=str.lower) #column name: lower letter
    tmp_df2['date'] = pd.to_datetime(tmp_df2.date)
    tmp_df2['date'] = tmp_df2['date'].dt.date
    tmp_df2['date'] = pd.to_datetime(tmp_df2.date) #datetime type
    
    res = pd.DataFrame(tmp_df2.groupby('date')['dailyaveragewindspeed'].mean()).reset_index() #groupby date to form daily data
    return res

In [36]:
def clean_weather_data_hourly(weather_df):
    # add columns related to hourly data
#     df = pd.merge(weather_df['DATE'], weather_df['DailySustainedWindSpeed'], left_index=True, right_index=True)
#     df1 = pd.merge(weather_df['DailyPrecipitation'], weather_df['HourlyPrecipitation'], left_index=True, right_index=True)
#     df2 = pd.merge(df, df1, left_index=True, right_index=True)
    
    df2 = weather_df[['DATE', 'DailySustainedWindSpeed', 'DailyPrecipitation', 'HourlyPrecipitation']]
    
    df2.loc[df2['HourlyPrecipitation']=='T','HourlyPrecipitation']=0 #T to 0
    df2 = df2.drop(df2[df2['HourlyPrecipitation'].str.contains(pat='s')==True].index) #data including 1.2s 
    df2['HourlyPrecipitation']=df2['HourlyPrecipitation'].astype(float) #to float 
    df2.loc[df2['DailyPrecipitation'] == 'T','DailyPrecipitation'] = 0 
    df2['DailyPrecipitation'] = pd.to_numeric(df2['DailyPrecipitation'], errors='coerce')
    df2 = df2.rename(columns=str.lower)
    df2['date']=pd.to_datetime(df2.date)
    return df2

In [37]:
daily = clean_weather_data_daily(weather_dfs)
daily

,date,dailyaveragewindspeed
0,2009-01-01,NaN
1,2009-01-02,NaN
2,2009-01-03,NaN
3,2009-01-04,NaN
4,2009-01-05,NaN
...,...,...
2546,2015-12-27,5.7
2547,2015-12-28,8.3
2548,2015-12-29,7.0
2549,2015-12-30,4.1


In [38]:
def load_and_clean_weather_data(weather_dfs):
    # hourly data cleaning
#     hourly_dataframes = []
    hour = pd.DataFrame(clean_weather_data_hourly(weather_dfs)).reset_index()
#     for csv_file in weather_dfs:
#         hourly_dataframe = clean_weather_data_hourly(csv_file)  
#         hourly_dataframes.append(hourly_dataframe)

#     # daily data cleaning
#     weather14 =weather_dfs['dailyaveragewindspeed'].dropna(axis='column')
#     daily14 = clean_weather_data_daily(weather14)

#     # adjust for missing data
#     average = daily14[daily14.index >= datetime.strptime('2014-01-25', '%Y-%m-%d')][:3]['dailyaveragewindspeed'].mean()
#     daily14.loc[daily14.index == '2014-01-26', 'dailyaveragewindspeed'] = average

#     # use cleaned daily data
#     daily = daily14

    # hourly data cleaning
    # hour = pd.concat(hourly_dataframes)
    hour['dateh'] = hour['date'].dt.strftime('%Y-%m-%d %H')
    hour = hour.groupby(['dateh']).agg({'date': 'last', 'dailysustainedwindspeed': 'mean',
                                         'dailyprecipitation': 'mean', 'hourlyprecipitation': 'mean'})

    # hourly precipitation
    # 1. Use daily precipitation /24 to estimate missing hourly precipitation
    hour.loc[hour.hourlyprecipitation.isnull() == True, 'hourlyprecipitation'] = \
        hour.loc[hour.hourlyprecipitation.isnull() == True, 'dailyprecipitation'] / 24

    # 2. use average of windows to fill the missing value
    roll = pd.DataFrame(hour.hourlyprecipitation.rolling(window=24, min_periods=1).mean())
    roll.columns = ['hourlyprecipitation_roll']
    hour = pd.merge(hour, roll, left_index=True, right_index=True)

    hour.loc[((hour['hourlyprecipitation'].isnull() == True) &
              (hour['hourlyprecipitation_roll'].isnull() == False)),
             'hourlyprecipitation'] = hour.loc[((hour['hourlyprecipitation'].isnull() == True) &
                                                (hour['hourlyprecipitation_roll'].isnull() == False)),
                                               'hourlyprecipitation_roll']

    # 3. the rest fill with 0
    hour.loc[(hour['hourlyprecipitation'].isnull() == True), 'hourlyprecipitation'] = 0
    del hour['dailyprecipitation'], hour['hourlyprecipitation_roll']

    # sustained wind speed
    hour['ymd'] = hour.date.dt.strftime('%Y-%m-%d')
    new = pd.DataFrame(hour.groupby('ymd')['dailysustainedwindspeed'].mean())
    new.columns = ['sustainedwindspeed']
    new = new.reset_index()
    hour = pd.merge(hour, new, on='ymd')
    del hour['dailysustainedwindspeed'], hour['ymd']

    hour = hour.reset_index(drop=True) 
    daily1 = daily.reset_index() 
    daily1['date'] = daily['date'].dt.date
    daily1 = daily1.drop(columns = 'index')

    return hour, daily1

weather_hourly = load_and_clean_weather_data(weather_dfs)[0]
weather_daily = load_and_clean_weather_data(weather_dfs)[1]

In [39]:
weather_hourly

,date,hourlyprecipitation,sustainedwindspeed
0,2009-01-01 00:51:00,0.0,NaN
1,2009-01-01 01:51:00,0.0,NaN
2,2009-01-01 02:51:00,0.0,NaN
3,2009-01-01 03:51:00,0.0,NaN
4,2009-01-01 04:51:00,0.0,NaN
...,...,...,...
60436,2015-12-31 19:51:00,0.0,14.0
60437,2015-12-31 20:51:00,0.0,14.0
60438,2015-12-31 21:51:00,0.0,14.0
60439,2015-12-31 22:51:00,0.0,14.0


In [40]:
weather_daily

,date,dailyaveragewindspeed
0,2009-01-01,NaN
1,2009-01-02,NaN
2,2009-01-03,NaN
3,2009-01-04,NaN
4,2009-01-05,NaN
...,...,...
2546,2015-12-27,5.7
2547,2015-12-28,8.3
2548,2015-12-29,7.0
2549,2015-12-30,4.1


### 1.3 Preprocessing Uber Rides data

In [41]:
# read samples for uber rides
uber_rides = pd.read_csv('uber_rides_sample.csv')

In [42]:
len(uber_rides)

200000

In [43]:
uber_rides

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1


In [44]:
# datetime processing
uber_rides = uber_rides.iloc[:,2:] 
uber_rides["pickup_datetime"]=pd.to_datetime(uber_rides["pickup_datetime"]) 
uber_rides['pickup_datetime']=uber_rides['pickup_datetime'].dt.tz_localize(None)
# pd.to_datetime(uber['pickup_datetime'].dt.date)

In [45]:
# date to time columns
def set_times(renamed_data):
    renamed_data["year"]=renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").year )
    renamed_data["month"]=renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").month )
    renamed_data["day"]=renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").day )
    renamed_data["hour"]=renamed_data["pickup_datetime"].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").hour )
    renamed_data['week']=renamed_data['pickup_datetime'].dt.dayofweek + 1 
    return renamed_data

set_times(uber_rides)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,week
0,7.5,2015-05-07 19:52:06,-73.999817,40.738354,-73.999512,40.723217,1,2015,5,7,19,4
1,7.7,2009-07-17 20:04:56,-73.994355,40.728225,-73.994710,40.750325,1,2009,7,17,20,5
2,12.9,2009-08-24 21:45:00,-74.005043,40.740770,-73.962565,40.772647,1,2009,8,24,21,1
3,5.3,2009-06-26 08:22:21,-73.976124,40.790844,-73.965316,40.803349,3,2009,6,26,8,5
4,16.0,2014-08-28 17:47:00,-73.925023,40.744085,-73.973082,40.761247,5,2014,8,28,17,4
...,...,...,...,...,...,...,...,...,...,...,...,...
199995,3.0,2012-10-28 10:49:00,-73.987042,40.739367,-73.986525,40.740297,1,2012,10,28,10,7
199996,7.5,2014-03-14 01:09:00,-73.984722,40.736837,-74.006672,40.739620,1,2014,3,14,1,5
199997,30.9,2009-06-29 00:42:00,-73.986017,40.756487,-73.858957,40.692588,2,2009,6,29,0,1
199998,14.5,2015-05-20 14:56:25,-73.997124,40.725452,-73.983215,40.695415,1,2015,5,20,14,3


In [46]:
# for uber trips
## calculate distance between pickup and dropoff
## by using the Haversine Formula
def to_dist(lat1, lon1, lat2, lon2):    
    R = 6371  # radius of the Earth in kilometers
    dLat = lat2.apply(math.radians) - lat1.apply(math.radians)
    dLon = lon2.apply(math.radians) - lon1.apply(math.radians)
    lat1 = lat1.apply(math.radians)
    lat2 = lat2.apply(math.radians)
    a = (dLat / 2).apply(math.sin) ** 2 + (dLat / 2).apply(math.sin) ** 2 * lat1.apply(math.cos) * lat2.apply(math.cos)
    c = 2 * np.arctan2(a.apply(math.sqrt), (1 - a).apply(math.sqrt))    
    return R * c

# cleaned_df["distance"] = to_dist(cleaned_df["pickup_latitude"], cleaned_df["pickup_longitude"], cleaned_df["dropoff_latitude"], cleaned_df["dropoff_longitude"])
uber_rides["distance"] = to_dist(uber_rides["pickup_latitude"], uber_rides["pickup_longitude"], uber_rides["dropoff_latitude"], uber_rides["dropoff_longitude"])
uber_rides  

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,week,distance
0,7.5,2015-05-07 19:52:06,-73.999817,40.738354,-73.999512,40.723217,1,2015,5,7,19,4,2.111795
1,7.7,2009-07-17 20:04:56,-73.994355,40.728225,-73.994710,40.750325,1,2009,7,17,20,5,3.083131
2,12.9,2009-08-24 21:45:00,-74.005043,40.740770,-73.962565,40.772647,1,2009,8,24,21,1,4.446678
3,5.3,2009-06-26 08:22:21,-73.976124,40.790844,-73.965316,40.803349,3,2009,6,26,8,5,1.743997
4,16.0,2014-08-28 17:47:00,-73.925023,40.744085,-73.973082,40.761247,5,2014,8,28,17,4,2.394064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,3.0,2012-10-28 10:49:00,-73.987042,40.739367,-73.986525,40.740297,1,2012,10,28,10,7,0.129742
199996,7.5,2014-03-14 01:09:00,-73.984722,40.736837,-74.006672,40.739620,1,2014,3,14,1,5,0.388254
199997,30.9,2009-06-29 00:42:00,-73.986017,40.756487,-73.858957,40.692588,2,2009,6,29,0,1,8.915154
199998,14.5,2015-05-20 14:56:25,-73.997124,40.725452,-73.983215,40.695415,1,2015,5,20,14,3,4.191060
